In [1]:
import requests
import pandas as pd
import json
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd
import json
from pandas import json_normalize
import plotly.express as px
import plotly.io as pio
from functools import reduce

In [2]:
villes=["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille", "Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [3]:
# Treatment in order to standardize the names
nvelle_liste_ville=[]
for ville in villes:
    nvelle_ville=ville.lower().replace(" ","+")
    nvelle_liste_ville.append(nvelle_ville)

In [45]:
# Define iteration to get the the wheather for the 35 cities from GPS coordonates and add all the information in a list named meteo
meteo= []
for city in villes:
    r = requests.get(f'https://nominatim.openstreetmap.org/search?q={city}&format=json') 
    lat = r.json()[0]['lat']
    lon = r.json()[0]['lon']
    for i in range(40):
        r2 = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid=b8ddc185d198199709ed2cb5171bc980&lang=fr&units=metric')
        temp = r2.json()['list'][i]['main']['temp']
        date=r2.json()['list'][i]['dt_txt']
        meteo.append([city,date,temp, lon,lat])

In [46]:
# We save our resuest results in a json file named meteo
with open('meteo.json', 'w') as f:
    json.dump(meteo, f)

In [59]:
df=pd.DataFrame(meteo, columns=['city','date','temperature', 'lon','lat'])

In [60]:
# We use the library datetime on our date column in order to split it on Day & hour
df['date']=pd.to_datetime(df['date'])
df["day"]=df.date.dt.day
df["Hour"]=df.date.dt.hour
df.day=df.day.apply(lambda x:'day1' if x==18
                    else 'day2' if x==19 
                    else 'day3' if x==20
                    else 'day4' if x==21
                    else 'day5' if x==22
                    else 'day6')
df.head()

,city,date,temperature,lon,lat,day,Hour
0,Mont Saint Michel,2023-02-18 21:00:00,11.39,-1.511459954959514,48.6359541,day1,21
1,Mont Saint Michel,2023-02-19 00:00:00,11.13,-1.511459954959514,48.6359541,day2,0
2,Mont Saint Michel,2023-02-19 03:00:00,9.42,-1.511459954959514,48.6359541,day2,3
3,Mont Saint Michel,2023-02-19 06:00:00,8.20,-1.511459954959514,48.6359541,day2,6
4,Mont Saint Michel,2023-02-19 09:00:00,9.56,-1.511459954959514,48.6359541,day2,9


In [57]:
# we reate a function to format the city names, keeping only the first part before the comma of the whole name
def get_name(name):
    return name.split(',')[0]

In [ ]:
"""# we apply this function to our first name column
df['city']=df.loc[:,'city'].apply(get_name)
df.head()"""

In [69]:
# We apply some text modifications in order to standradize the city names
df.city=df.city.apply(lambda x: x.lower().replace("-"," ").replace("î", "i")).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df.city=df.city.apply(lambda x: x.replace("knigsbourg","koenigsbourg").replace("saint malo", "st malo"))

In [61]:
# We open the csv file created from our Booking.com scrapping
json_file_path = "ScrapBooking.json"

with open(json_file_path, 'r') as j:
     infos_booking = json.loads(j.read())

In [70]:
# we create from this a csv file a pandas dataframe named booking
booking=pd.DataFrame(infos_booking)

In [72]:
# We put all the letters in lowercase and replace all the - by a space
booking.city=booking.city.apply(lambda x: x.lower().replace("-"," "))

In [73]:
#We merge both dataframes
data=pd.merge(df, booking, how="outer", on="city")
data.head()

,city,date,temperature,lon,lat,day,Hour,name,url,score,description,gps
0,mont saint michel,2023-02-18 21:00:00,11.39,-1.511459954959514,48.6359541,day1,21,Au Mont Pour Deux,https://www.booking.com/hotel/fr/au-mont-pour-...,8.3,Managed by a private host,"48.61349776,-1.48584455"
1,mont saint michel,2023-02-18 21:00:00,11.39,-1.511459954959514,48.6359541,day1,21,Gites Bellevue,https://www.booking.com/hotel/fr/gites-bellevu...,9.3,Managed by a private host,"48.60788007,-1.51722372"
2,mont saint michel,2023-02-18 21:00:00,11.39,-1.511459954959514,48.6359541,day1,21,Chambres d'Hôtes Les Vieilles Digues,https://www.booking.com/hotel/fr/chambres-d-ha...,9.3,"Set in a renovated Breton-style house, this fa...","48.60418750,-1.51159912"
3,mont saint michel,2023-02-18 21:00:00,11.39,-1.511459954959514,48.6359541,day1,21,Maison au pied du Mont Saint Michel,https://www.booking.com/hotel/fr/maison-au-pie...,9.0,"Boasting garden views, Maison au pied du Mont ...","48.61267358,-1.48553065"
4,mont saint michel,2023-02-18 21:00:00,11.39,-1.511459954959514,48.6359541,day1,21,Au Mont Chez Nous Jaccuzi et Sauna,https://www.booking.com/hotel/fr/26-route-de-l...,9.2,Managed by a private host,"48.61345530,-1.48587230"


In [74]:
#Let's have a look to null values
data[data.isna().any(axis=1)]

,city,date,temperature,lon,lat,day,Hour,name,url,score,description,gps
12004,besancon,2023-02-18 21:00:00,9.13,6.0243622,47.2380222,day1,21,Spacieux 3 chambres refait à neuf,https://www.booking.com/hotel/fr/spacieux-3-ch...,None,Managed by a private host,"47.24860190,6.03051780"
12007,besancon,2023-02-18 21:00:00,9.13,6.0243622,47.2380222,day1,21,"Le Bellevue - Parc, Gare et Vieille ville",https://www.booking.com/hotel/fr/le-bellevue-p...,None,Managed by a private host,"47.24595380,6.02309120"
12029,besancon,2023-02-19 00:00:00,8.79,6.0243622,47.2380222,day2,0,Spacieux 3 chambres refait à neuf,https://www.booking.com/hotel/fr/spacieux-3-ch...,None,Managed by a private host,"47.24860190,6.03051780"
12032,besancon,2023-02-19 00:00:00,8.79,6.0243622,47.2380222,day2,0,"Le Bellevue - Parc, Gare et Vieille ville",https://www.booking.com/hotel/fr/le-bellevue-p...,None,Managed by a private host,"47.24595380,6.02309120"
12054,besancon,2023-02-19 03:00:00,7.88,6.0243622,47.2380222,day2,3,Spacieux 3 chambres refait à neuf,https://www.booking.com/hotel/fr/spacieux-3-ch...,None,Managed by a private host,"47.24860190,6.03051780"
...,...,...,...,...,...,...,...,...,...,...,...,...
33890,bayonne,2023-02-23 06:00:00,7.51,-1.4736657,43.4945144,day6,6,Le Petit Nid Douillet de Jéjé,https://www.booking.com/hotel/fr/le-petit-nid-...,None,Managed by a private host,"43.49405931,-1.46824640"
33915,bayonne,2023-02-23 09:00:00,8.04,-1.4736657,43.4945144,day6,9,Le Petit Nid Douillet de Jéjé,https://www.booking.com/hotel/fr/le-petit-nid-...,None,Managed by a private host,"43.49405931,-1.46824640"
33940,bayonne,2023-02-23 12:00:00,8.72,-1.4736657,43.4945144,day6,12,Le Petit Nid Douillet de Jéjé,https://www.booking.com/hotel/fr/le-petit-nid-...,None,Managed by a private host,"43.49405931,-1.46824640"
33965,bayonne,2023-02-23 15:00:00,8.50,-1.4736657,43.4945144,day6,15,Le Petit Nid Douillet de Jéjé,https://www.booking.com/hotel/fr/le-petit-nid-...,None,Managed by a private host,"43.49405931,-1.46824640"


In [76]:
# Removing lines with nan values from score column
data = data[data.score.isna() == False]

In [78]:
# We create a csv file from this dataframe
kayakdata = data.to_csv()

# Create a Data Lake using AWS S3

In [79]:
#connexion and Bucket S3 creation on AWS
import boto3

session = boto3.Session(profile_name="ferielprojet")
s3 = session.resource("s3")


In [87]:
response = s3.create_bucket(
    Bucket='projetkayak-feriel',
    CreateBucketConfiguration={
        'LocationConstraint': 'eu-west-3',
    },
)

In [80]:
bucket = s3.Bucket("projetkayak-feriel") 

In [81]:
# we add our csv file to this new empty bucket
put_object = bucket.put_object(Key="kayakdata.csv", Body=kayakdata)

In [82]:
# we Download from s3 bucket 
bucket.download_file('kayakdata.csv', 'kayak.csv')

# ETL PROCESS

## Cleaning

In [232]:
# Convert out csv file previously downloded to a pandas Dataframe
df=pd.read_csv('kayak.csv')
df.head()

,Unnamed: 0,city,date,temperature,lon,lat,day,Hour,name,url,score,description,gps
0,0,mont saint michel,2023-02-18 21:00:00,11.39,-1.51146,48.635954,day1,21,Au Mont Pour Deux,https://www.booking.com/hotel/fr/au-mont-pour-...,8.3,Managed by a private host,"48.61349776,-1.48584455"
1,1,mont saint michel,2023-02-18 21:00:00,11.39,-1.51146,48.635954,day1,21,Gites Bellevue,https://www.booking.com/hotel/fr/gites-bellevu...,9.3,Managed by a private host,"48.60788007,-1.51722372"
2,2,mont saint michel,2023-02-18 21:00:00,11.39,-1.51146,48.635954,day1,21,Chambres d'Hôtes Les Vieilles Digues,https://www.booking.com/hotel/fr/chambres-d-ha...,9.3,"Set in a renovated Breton-style house, this fa...","48.60418750,-1.51159912"
3,3,mont saint michel,2023-02-18 21:00:00,11.39,-1.51146,48.635954,day1,21,Maison au pied du Mont Saint Michel,https://www.booking.com/hotel/fr/maison-au-pie...,9.0,"Boasting garden views, Maison au pied du Mont ...","48.61267358,-1.48553065"
4,4,mont saint michel,2023-02-18 21:00:00,11.39,-1.51146,48.635954,day1,21,Au Mont Chez Nous Jaccuzi et Sauna,https://www.booking.com/hotel/fr/26-route-de-l...,9.2,Managed by a private host,"48.61345530,-1.48587230"


In [233]:
# Droping useless columns
df=df.drop(["Unnamed: 0"], axis=1)

In [234]:
# Create a dataframe with only our hotels
df_hotels= df[['city','name','score','description','url','lat','lon', 'gps']].drop_duplicates(keep='first')


In [236]:
df_hotels['gps']=df_hotels.gps.apply(lambda x:x.split(','))
df_hotels['hotel_lat']=df_hotels.gps.apply(lambda x:x[0])
df_hotels['hotel_lon']= df_hotels.gps.apply(lambda x:x[1])

In [88]:
# Create dataframe weather 
df_weather= df[['city','temperature','lat','lon', 'day','Hour']]

## Using cleaned data to create a Data Warehouse

In [77]:
## Libraries to install if other connections than sqlite
# !pip install pymysql # For MySQL engines
# !pip install psycopg2-binary # For PostgreSQL engines

In [2]:
import sqlalchemy as db
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker 
import psycopg2

In [3]:
# Define database connection variables
DBHOST = "kayakferiel.cojmtfidq16h.eu-west-3.rds.amazonaws.com"
DBUSER = "postgres"
DBPASS = "kayakmedjaoui"
DBNAME = ""
PORT = "5432"
REGION = "eu-west-3b"

In [4]:
# Replace YOUR_USERNAME, YOUR_PASSWORD and YOUR_HOSTNAME
# by your credentials
engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

In [5]:
# Initialize a sessionmaker 
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)


In [241]:
# Create tables on RDS from our dataframes
df_hotels.to_sql("hotel", engine)
df_weather.to_sql("weather", engine)

2023-02-18 22:19:01,281 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-02-18 22:19:01,282 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-18 22:19:01,295 INFO sqlalchemy.engine.Engine select current_schema()
2023-02-18 22:19:01,296 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-18 22:19:01,309 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-02-18 22:19:01,310 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-18 22:19:01,327 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-02-18 22:19:01,332 INFO sqlalchemy.engine.Engine [generated in 0.00526s] {'name': 'hotel'}
2023-02-18 22:19:01,356 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-18 22:19:01,361 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotel (
	index BIGINT, 
	city TEXT, 
	name TEXT, 
	score FLOAT(53), 
	description TEXT, 
	url TEXT, 
	lat FLOAT(53), 
	lon 

860

In [6]:
# SQL request to get all the information of the table weather & create a panda dataframe with the results
best_cities_weather_sql= text("SELECT * FROM weather")
best_cities_weather = pd.read_sql(best_cities_weather_sql, engine).drop('index', axis=1)

2023-02-23 19:26:25,400 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-02-23 19:26:25,406 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 19:26:25,417 INFO sqlalchemy.engine.Engine select current_schema()
2023-02-23 19:26:25,421 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 19:26:25,433 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-02-23 19:26:25,435 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-02-23 19:26:25,452 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-02-23 19:26:25,453 INFO sqlalchemy.engine.Engine [generated in 0.00102s] {'name': 'SELECT * FROM weather'}
2023-02-23 19:26:25,473 INFO sqlalchemy.engine.Engine SELECT * FROM weather
2023-02-23 19:26:25,474 INFO sqlalchemy.engine.Engine [generated in 0.00159s] {}


In [7]:
# We create a dataframe with the temperature mean for each city in a dicreasing order
best_cities_weather = best_cities_weather.groupby(by='city', as_index = False).mean().sort_values(by = 'temperature',ascending = False)

In [8]:
# We keep only the 5 first cities and create a list with their names
cities=best_cities_weather.head(5)
Top5_cities=cities.city.tolist()

In [9]:
# SQL request to get all the information of the table hotel & create a panda dataframe with the results
best_hotels_sql = text("SELECT * FROM hotel ")
best_hotels = pd.read_sql(best_hotels_sql, engine).drop('index', axis=1)

2023-02-23 19:26:40,254 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-02-23 19:26:40,267 INFO sqlalchemy.engine.Engine [cached since 14.82s ago] {'name': 'SELECT * FROM hotel '}
2023-02-23 19:26:40,292 INFO sqlalchemy.engine.Engine SELECT * FROM hotel 
2023-02-23 19:26:40,304 INFO sqlalchemy.engine.Engine [generated in 0.01138s] {}


In [10]:
#We keep only the hotels located in a city of our top 5 cities we sort the dataframe by city and score of the hotel on booking
selected_hotels= best_hotels[best_hotels["city"].isin (Top5_cities)].sort_values(by = ['city','score'],ascending = False)

In [11]:
#we create a dataframe for each city and 20 best hotels in order to create a map
hotel_city1=selected_hotels[selected_hotels['city']==Top5_cities[0]].head(20)
hotel_city2=selected_hotels[selected_hotels['city']==Top5_cities[1]].head(20)
hotel_city3=selected_hotels[selected_hotels['city']==Top5_cities[2]].head(20)
hotel_city4=selected_hotels[selected_hotels['city']==Top5_cities[3]].head(20)
hotel_city5=selected_hotels[selected_hotels['city']==Top5_cities[4]].head(20)

In [12]:
# We concatenate all previous dataframe into one
all_hotels=pd.concat([hotel_city1, hotel_city2,hotel_city3,hotel_city4,hotel_city5], axis=0)

## Vizualisations

Weather vizualisation

In [17]:
# Display weather map base on the higher temperatures
fig = px.scatter_mapbox(cities,
                        title='TOP 5 cities in France',
                        lat = 'lat',
                        lon = 'lon',
                        size_max = 20,
                        hover_name = 'city',
                        color='temperature',
                        size='temperature'
                        )
fig.update_layout(width = 800,
                  height = 600,
                  title_x = 0.5, 
                  template = 'plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "zoom": 4},
                 plot_bgcolor='#2d3035', paper_bgcolor='#2d3035')

                  
                

fig.show()

In [19]:
# We transform score, hotel longitude and latitude to numeric data for our next steps
all_hotels["score"] = pd.to_numeric(all_hotels["score"])
all_hotels["hotel_lat"] = pd.to_numeric(all_hotels["hotel_lat"]).round(decimals=6)
all_hotels["hotel_lon"] = pd.to_numeric(all_hotels["hotel_lon"]).round(decimals=6)

In [24]:
# Display Top 20 best hotels (based on their score) located in our TOP 5 cities
fig = px.scatter_mapbox(all_hotels,
                        title='TOP 20 hotels in Top 5 cities in France',
                        lat = 'hotel_lat',
                        lon = 'hotel_lon',
                        color='score',
                        size='score',
                        color_continuous_scale=px.colors.sequential.Inferno,
                        size_max=10,
                        hover_name='name',
                        hover_data={
                            'description':True,
                            'name': True,
                            'score': True    
                            }, 
                                            )
fig.update_layout(width = 800,
                  height = 600,
                  title_x = 0.5, 
                  template = 'plotly_dark',
                  mapbox = {"style": "carto-darkmatter", "zoom": 6},
                  
                  )

fig.show()